# Chapter 6. 결정 트리

* 결정 트리(decision tree)
 * 분류와 회귀 작업 그리고 다중출력 작업도 가능한 머신러닝 알고리즘
 * 최근 사용되는 강력한 머신러닝 알고리즘 중 하나인 랜덤 포레스트(7장 참조)의 기본 구성 요소

## Set Up

In [12]:
import os

# Where to save the figures
PROJECT_ROOT_DIR = "."

def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, "images", fig_id)

## 6.1 결정 트리 학습과 시각화

In [13]:
from sklearn.datasets import load_iris # 붓꽃 데이터셋
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # 꽃잎의 길이와 너비
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth = 2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [14]:
from sklearn.tree import export_graphviz

# export_graphviz() 함수를 사용해 그래프 정의를 iris_tree.dot 파일로 출력하여 시각화

export_graphviz(
    tree_clf,
    out_file = image_path("iris_tree.dot"),
    feature_names = iris.feature_names[2:],
    class_names = iris.target_names,
    rounded = True,
    filled = True
)

In [17]:
# brew install graphviz
# 붓꽃 결정 트리 시각화
! dot -Tpng images/iris_tree.dot -o images/iris_tree.png

## 6.2 예측하기

결정 트리의 장점 중 하나는 데이터 전처리가 거의 필요하지 않다는 것. 특성의 스케일을 맞추거나 평균을 원점에 맞추는 직업이 필요하지 않다.

### 지니 불순도 (gini impurity)
* 한 노드의 모든 샘플이 같은 클래스 속에 속해 있다면 이 노드를 순수(gini=0)하다고 한다.

### 사이킷런
* 사이킷런은 이진 트리만 만드는 CART 알고리즘을 사용한다.
 * 리프 노드 외의 모든 녿드는 자식 노드를 2개씩 가진다. 즉, 질문의 답은 '예' 혹은 '아니오'
* ID3 같은 알고리즘은 둘 이상의 자식 노드를 가진 결정 트리를 만들 수 있다.

### 화이트박스와 블랙박스
* 화이트박스
 * 직관적이고 결정 방식을 이해하기 쉬움
 * e.g. 결정 트리
* 블랙박스
 * 성능이 뛰어나고 예측을 만드는 연산 과정을 쉽게 확인 가능
 * but 왜 그런 예측을 만드는지 쉽게 설명하기 어려움
 * e.g. 랜덤 포레스트, 신경망
 
결정 트리는 필요하다면 수동으로 직접 따라 해볼 수도 있는 간단하고 명확한 분류 방법 사용

## 6.3 클래스 확률 추정

In [18]:
tree_clf.predict_proba([[5, 1.5]]) # 길이가 5cm이고 너비가 1.5cm인 꽃잎의 각 클래스 별 속할 확률 추정

array([[0.        , 0.90740741, 0.09259259]])

In [19]:
tree_clf.predict([[5, 1.5]]) # 길이가 5cm이고 너비가 1.5cm인 꽃잎의 클래스 예측

array([1])

## 6.4 CART 훈련 알고리즘

사이킷런은 결정 트리를 훈련(성장)시키기 위해 CART<sup>classification and regression tree</sup> 알고리즘을 사용한다.

### CART 알고리즘
* 탐욕적 알고리즘
 * 최적의 솔루션을 보장하지는 않음
* 최적의 트리를 찾는 것은 NP-완전<sup>NP-Complete</sup> 문제로 알려져 있음
 * *O(exp(m))* 시간 필요
 * 매우 작은 훈련 세트에도 적용하기 어려움
 * 따라서 '납득할 만한 좋은 솔루션'으로만 만족해야 함

## 6.5 계산 복잡도

결정 트리는 예측을 위해 루트 노드에서부터 리프 노드까지 탐색해야 한다. 일반적으로 거의 균형을 이루고 있어 탐색할 때 약 *O(log<sub>2</sub>(m))* 개의 노드를 거쳐야 한다.
각 노드는 하나의 특성값만 확인하기 때문에 예측에 필요한 전체 복잡도는 특성 수와 무관하게 *O(log<sub>2</sub>(m))* 이다. 큰 훈련 세트를 다룰 때도 예측 속도가 매우 빠르다.

**훈련 알고리즘**은 각 노드에서 모든 훈련 샘플의 모든 특성을 비교하므로 훈련 복잡도는 *O(n x m log<sub>2</sub>(m))* 이 된다.
훈련 세트가 (수천 개 이하의 샘플 정도로) 작을 경우 사이킷런은 (presort = True로 지정하면) 미리 데이터를 정렬하여 훈련 속도를 높일 수 있다.
 * 성능 향상이 크지 않아 사이킷런 0.24 버전에서는 결정 트리와 그레이디언트 부스팅 클래스의 presort 매개변수가 삭제되었다.

훈련 세트가 클 경우에는 속도가 많이 느려진다.

## 6.6 지니 불순도 또는 엔트로피?

기본적으로 지니 불순도가 사용되지만 creterion 매개변수를 "entropy"로 지정하여 엔트로피 불순도를 사용할 수 있다.

### 엔트로피
* 분자의 무질서함을 측정하는 것
* 원래 열역학의 개념
* 분자가 안정되고 질서 정연하면 엔트로피가 0에 가깝다
* 머신러닝의 경우 어떤 세트가 한 클래스의 샘플만 담고 있다면 엔트로피가 0이다.

지니 불순도와 엔트로피는 큰 차이는 없으나(비슷한 트리를 만들어내나) 지니 불순도가 조금 더 계산이 빠르기 때문에 기본값으로 좋다.
다른 트리가 만들어지는 경우 지니 불순도가 가장 빈도 높은 클래스를 한쪽 가지로 고립시키는 경향이 있는 반면 엔트로피는 조금 더 균형 잡힌 트리를 만든다.

## 6.7 규제 매개변수

결정 트리는 훈련 데이터에 대한 제약 사항이 거의 없다. 훈련되기 전에 파라미터 수가 결정되지 않는 **비파라미터 모델**로, 모델 구조가 데이터에 맞춰져서 고정되지 않고 자유롭다. 제한을 두지 않으면 대부분 과대적합되기 쉽다.

반대로 선형 모델은 데이터가 선형일거라 가정한다. 미리 정의된 모델 파라미터 수를 가지는 **파라미터 모델**로, 자유도가 제한되고 과대적합될 위험이 줄어든다. (과소적합 위험은 커진다.)

과대적합을 피하기 위해 결정 트리의 자유도를 제한해야 하며 이를 **규제**라고 한다.
사이킷런에서는 max_depth 매개변수로 결정 트리의 최대 깊이를 제어한다. (기본값은 None, 제한 없음)
max_depth를 줄이면 모델을 규제하게 되고 과대적합의 위험이 감소한다.

* DecisionTreeClassifier
 * min_ 으로 시작하는 매개변수를 증가시키거나 max_ 로 시작하는 매개변수를 감소시키면 모델에 규제가 커진다.
 * `min_samples_split` 분할되기 위해 노드가 가져야 하는 최소 샘플 수
 * `min_samples_leaf` 리프 노드가 가지고 있어야 할 최소 샘플 수
 * `min_weight_fraction_leaf` min_samples_leaf와 동일하나 가중치가 부여된 점체 샘플 수에서의 비율
 * `max_leaf_nodes` 리프 노드의 최대 수
 * `max_features` 각 노드에서 분할에 사용할 특성의 최대 수

### 가지치기
* 제한 없이 결정 트리를 훈련시키고 불필요한 노드를 가지치기<sup>pruning</sup>하는 알고리즘도 존재
* 순도를 높이는 것이 통계적으로 큰 효과가 없다면 리프 노드 바로 위의 노드는 불필요할 수 있다
* 대표적으로 귀무가설<sup>null hypothesis</sup>
 * 검정<sup>chi-squared test</sup> 같은 통계적 검정을 사용하여 우연히 향상된 것인지 추정
 * 이 확률을 *p-값* 이라 부른다
 * 어떤 임곗값(보통 5%)보다 높으면 그 노드는 불필요한 것으로 간주되고 그 자식 노드는 삭제된다.
* 가지치기는 불필요한 노드가 모두 없어질 때까지 계속된다.

## 6.8 회귀

In [20]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth = 2)
tree_reg.fit(X, y)

DecisionTreeRegressor(max_depth=2)

회귀 결정 트리의 경우 분류 트리와 달리 각 노드에서 클래스를 예측하는 대신 어떤 값을 예측한다.

CART 알고리즘은 훈련 세트를 불순도를 최소화하는 방향으로 분할하는 대신 평균제곱오차(MSE)를 최소화하도록 분할한다. (그 외는 앞의 결정 트리와 비슷하게 작동한다)

분류에서와 같이 회귀 작업에서도 결정 트리가 과대적합되기 쉬우므로, 규제 매개변수를 사용한다.

## 6.9 불안정성

결정 트리는 이해하고 해석하기 쉬우며, 사용하기 편하고, 여러 용도로 사용할 수 있으며, 성능도 뛰어나다.

### 제약사항
결정 트리에는 몇가지 제한 사항이 존재한다.
* 결정 트리는 계단 모양의 결정 경계를 만든다. (모든 분할은 축에 수직이다) 따라서 훈련 세트의 회전에 민감하다.
 * 이에 대한 해결법 중 하나는 훈련 데이터를 더 좋은 방향으로 회전시키는 PCA 기법을 사용하는 것이다.
* 훈련 데이터에 있는 작은 변화에도 매우 민감하다.
 * 사이킷런에서 사용하는 훈련 알고리즘은 확률적이기 때문에 (random_state 매개변수 지정 없이는) 같은 훈련 데이터에서도 다른 모델을 얻을 수 있다.